In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('../data/transformed/transformed_complete_data.parquet')

In [ ]:
df_one_loc= df[df['PU_location']==43].reset_index(drop=True)
df_one_loc

In [ ]:
def get_features_and_target_indices(
        rides : pd.DataFrame,
        features :int,
        step_size :int
) -> pd.DataFrame:
    features_and_target_indices = []

    stop_position = len(rides) - 1
    feature_index = features
    target_index = features + 1
    starting_index = 0

    while target_index <= stop_position:
        features_and_target_indices.append((starting_index,feature_index,target_index))

        starting_index += step_size
        feature_index +=step_size
        target_index +=step_size

    return features_and_target_indices

In [ ]:
n_features = 12
step_size = 1
features_and_target = get_features_and_target_indices(df_one_loc,n_features,step_size)

In [ ]:
features_and_target[:5]

In [ ]:
df_one_loc.head(16)

In [10]:
from numpy.distutils.misc_util import is_sequence

C:\Users\user\AppData\Local\Temp\ipykernel_10836\3557039123.py:1: DeprecationWarning: 

  `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
  of the deprecation of `distutils` itself. It will be removed for
  Python >= 3.12. For older Python versions it will remain present.
  It is recommended to use `setuptools < 60.0` for those Python versions.
  For more details, see:
    https://numpy.org/devdocs/reference/distutils_status_migration.html 


  from numpy.distutils.misc_util import is_sequence


In [14]:
n_examples = len(features_and_target)
X = np.ndarray(shape=(n_examples,n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples),dtype=np.float32)
pu_hour = []


for i , idx in enumerate(features_and_target):
    X[i,:] = df_one_loc.iloc[idx[0]:idx[1]]['rides'].values
    
    random = df_one_loc.iloc[idx[1]:idx[2]]['rides'].values
    #print(is_sequence(random))
    if random.shape != (1,):
        print('position',i,random)
    y[i] = df_one_loc.iloc[idx[1]:idx[2]]['rides'].values
    
    
    pu_hour.append(df_one_loc.iloc[idx[1]]['PU_hours'])

In [ ]:
y

In [ ]:
features_df = pd.DataFrame(X,
columns= [f'rides_{x}_hr_before_target'.format(x) for x in reversed(range(1,n_features+1))]
                           )

In [ ]:
features_df.head()

In [ ]:
target_df = pd.DataFrame(y,columns=['Target_next_hour'])


In [ ]:
from tqdm import tqdm

complete_data = pd.read_parquet('../data/transformed/transformed_complete_data.parquet')



def model_features_and_target(
        ts_rides : pd.DataFrame,      
        n_features : int,
        step_size: int,
        
) -> pd.DataFrame:
        
        assert set(ts_rides.columns) == {'PU_hours','rides','PU_location'}
        location_ids = ts_rides['PU_location'].unique()
        
        features = pd.DataFrame()
        targets = pd.DataFrame()

        for loc_id in tqdm(location_ids):
                
                ts_rides = ts_rides[ts_rides['PU_location'] == loc_id].reset_index(drop = True)
                #ts_rides = ts_rides.loc[ts_rides.PU_location == loc_id,
                 #                       ['PU_hours','rides']]

                #get indices
                stop_position = len(ts_rides) - 1
                first_index = 0
                mid_indices = n_features
                last_index = n_features + 1
                feature_indices= []

                while last_index <= stop_position:
                        feature_indices.append((first_index,mid_indices,last_index))
                        first_index += step_size
                        mid_indices += step_size
                        last_index += step_size

               

                n_examples = len(feature_indices)
                X = np.ndarray(shape=(n_examples,n_features), dtype=np.float32)
                y = np.ndarray(shape=(n_examples),dtype=np.float32)
                pu_hour = []
               

                for i , idx in enumerate(feature_indices):
                        X[i,:] = ts_rides.iloc[idx[0]:idx[1]]['rides'].values
                        y[i] = ts_rides.iloc[idx[1]:idx[2]]['rides'].values
                        pu_hour.append(ts_rides.iloc[idx[1]]['PU_hours'])
                

                features_one_loc = pd.DataFrame(X ,
                                                columns=[f'rides_{x+1}_hr_before_target'.format(x) for x in reversed(range(n_features))])

                features_one_loc['PU_loc'] = loc_id
                features_one_loc['PU_hours'] = pu_hour

                features = pd.concat([features,features_one_loc])






                
                targets_one_loc = pd.DataFrame(y,columns=['next_hr_demand'])   
              
                targets = pd.concat([targets,targets_one_loc])
        
        features.reset_index(inplace=True,drop=True)
        targets.reset_index(inplace=True, drop=True)
        return features, targets   
                
features, targets = model_features_and_target(complete_data,24,1)

In [ ]:
features